In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [1]:
import json
import logging
import argparse
import numpy as np
import pandas as pd
import vllm
import torch
import string
import re
import collections
from sklearn.metrics import f1_score
import Levenshtein

In [11]:
#                          macro-f1/em          micro-f1/f1         
#                                 mean      std        mean      std
# (ner, matscholar)           80.12533  0.04045    84.62633  0.23798
# (ner, sofc_token)           78.55333  0.42465    89.54000  0.12165
# (ner, sc_comics)            89.47433  0.45713    92.46833  0.19805
# (pc, glass_non_glass)       90.29333  0.42272    91.77767  0.38509
# (sf, sofc_token)            75.95667  0.87344    85.36800  0.81667
# (ee, sc_comics)             90.79300  1.66148    93.44033  1.11710
# (re, structured_re)        100.00000  0.00000   100.00000  0.00000
# (re, sc_comics)             99.36867  0.10970    99.19300  0.14030
# (sar, synthesis_actions)    95.95367  0.15938    96.83567  0.23199
# (sc, sofc_sent)             76.74700  1.17814    93.37933  0.35192
# (qna, squad)                42.15267  0.21362    54.40733  0.69172

In [2]:
!$PATH

/bin/bash: /home/cse/btech/cs1200389/.conda/envs/matllama2.0/bin:/home/apps/anaconda3_2018/4.6.9/condabin:/home/soft/centOS/compilers/gcc/11.2/bin:/home/soft/cuda-11.0.2/bin:/home/soft/cuda-11.0.2/nvvm/bin:/usr/lib64/qt-3.3/bin:/usr/share/Modules/4.4.1/bin:/opt/pbs/default/bin:/opt/pbs/default/sbin:/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/pbs/2022.1.3/bin:/home/cse/btech/cs1200448/gurobi1003/linux64/bin:/home/cse/btech/cs1200448/bin: No such file or directory


In [3]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, LlamaTokenizer

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('--checkpoint', type=str, help="Path to the checkpoint to run inference on")
parser.add_argument('--valfile', type=str)
parser.add_argument('--multi_seed', action='store_true', help="Whether to run inference on multiple seeds or not.")
parser.add_argument('--batch_size', type=int, default=16, help="Batch size for inference")
args = parser.parse_args('')

In [11]:
args.checkpoint = "/scratch/cse/btech/cs1200448/MatLlama/ft-mthf/llama-original-squad"
args.valfile = '/scratch/cse/btech/cs1200448/MatLlama/ft_ds/val_ft.jsonl'

In [12]:
valfile = []
with open(args.valfile, 'r') as f:
    valfile = [json.loads(line) for line in f.readlines()]

In [13]:
# np.random.shuffle(valfile)
# with open(args.valfile, "w") as f:
#     for document in valfile:
#         f.write(json.dumps(document) + "\n")

In [14]:
valinputs = [f"<|im_start|>system\n{i['system']}<|im_end|>\n"+f"<|im_start|>question\n{i['question']}<|im_end|>\n"+"<|im_start|>answer\n" for i in valfile]

In [15]:
print(valinputs[0])

<|im_start|>system
You are a linguist and a material scientist. You need to mark the synthesis action for each of the keywords given after WORDS in the input. Answer to the question should be from one of the provided options. Do not output anything else other than the answer. You should output the word entity pair separated by ":" in each line. Your options are: cooling, heating, mixing, non-altering, purification, reaction, shaping, starting. Answer for each word must be in a new line.<|im_end|>
<|im_start|>question
WORDS: calcined
SENTENCE: The resulting solid was calcined at 500 ° C for 4 h in air with a heating rate of 10 ° C min-1 to obtain the final solid , which was denoted as CuCrOx .<|im_end|>
<|im_start|>answer



In [16]:
args.seed = 2

kwargs = {
    "model": args.checkpoint,
    "tokenizer": args.checkpoint,
    "trust_remote_code": True,
    "tensor_parallel_size": 1,
    "seed":args.seed,
    "gpu_memory_utilization":0.9,
}
client = vllm.LLM(**kwargs, dtype='float16')

INFO 04-05 22:36:34 llm_engine.py:87] Initializing an LLM engine with config: model='/scratch/cse/btech/cs1200448/MatLlama/ft-mthf/llama-original-squad', tokenizer='/scratch/cse/btech/cs1200448/MatLlama/ft-mthf/llama-original-squad', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=2)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-05 22:37:36 llm_engine.py:357] # GPU blocks: 1961, # CPU blocks: 512
INFO 04-05 22:37:38 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-05 22:37:38 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-05 22:37:47 model_runner.py:756] Graph capturing finished in 9 secs.


In [17]:
response = client.generate(valinputs[0]*1000, sampling_params=vllm.SamplingParams(
        best_of=1,
        presence_penalty=0.0,
        frequency_penalty=1.0,
        top_k=50,
        top_p=1.0,
        temperature=0.75,
        stop=["<|im_start|>", "<|im_end|>"],
        use_beam_search=False,
        max_tokens=500,
        logprobs=2
    ))

Processed prompts:   0%|                                                                                                                                                                                                              | 0/1 [00:00<?, ?it/s]

WARNING 04-05 22:38:56 scheduler.py:195] Input prompt (183001 tokens) is too long and exceeds limit of 2048


Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 268.56it/s]


In [19]:
response[0].outputs

[CompletionOutput(index=0, text='', token_ids=[], cumulative_logprob=0.0, logprobs=[], finish_reason=length)]

In [15]:
predictions += [""] + [i.outputs[0].text for i in response]

In [16]:
tasks = ["ner","pc","sf","ee","re","sar","sc"]
kw = {i : [] for i in tasks}
kw["ner"] = {i : [] for i in ["matscholar", "sofc_token", "sc_comics"]}
kw["pc"] = {i : [] for i in ["glass_non_glass"]}
kw["sf"] = {i : [] for i in ["sofc_token"]}
kw["ee"] = {i : [] for i in ["sc_comics"]}
kw["re"] = {i : [] for i in ["structured_re", "sc_comics"]}
kw["sar"] = {i : [] for i in ["synthesis_actions"]}
kw["sc"] = {i : [] for i in ["sofc_sent"]}

kw['ner']['matscholar'] = ['b-mat','i-mat','b-pro','i-pro','b-dsc','i-dsc','b-spl','i-spl','b-apl','i-apl','b-smt','i-smt','b-cmt','i-cmt']
kw['ner']['sofc_token'] = ['b-material', 'i-material', 'b-device', 'i-device', 'b-experiment', 'i-experiment', 'b-value', 'i-value']
kw['ner']['sc_comics'] = ['material', 'doping', 'sc', 'value', 'process', 'characterization', 'element', 'property', 'main']
kw['pc']['glass_non_glass'] = ['yes','no']
kw['sf']['sofc_token'] = ['i-device', 'b-voltage', 'b-anode_material', 'b-cathode_material', 'b-time_of_operation', 'i-working_temperature', 'b-conductivity', 'i-fuel_used', 'i-interlayer_material', 'i-time_of_operation', 'i-anode_material', 'i-current_density', 'b-degradation_rate', 'i-resistance', 'i-conductivity', 'b-current_density', 'b-working_temperature', 'i-thickness', 'i-experiment_evoking_word', 'b-open_circuit_voltage', 'i-degradation_rate', 'b-electrolyte_material', 'i-open_circuit_voltage', 'i-electrolyte_material', 'b-fuel_used', 'b-power_density', 'i-power_density', 'b-interlayer_material', 'b-thickness', 'b-device', 'b-experiment_evoking_word', 'i-cathode_material', 'b-resistance', 'i-support_material', 'i-voltage', 'b-support_material']
kw['ee']['sc_comics'] = ['site', 'dopant']
kw['re']['structured_re'] = ['capacity','voltage','coulombic efficiency','conductivity','energy']
kw['re']['sc_comics'] = ['target', 'condition', 'equivalent']
kw['sar']['synthesis_actions'] = ['cooling', 'heating', 'mixing', 'non-altering', 'purification', 'reaction', 'shaping', 'starting']
kw['sc']['sofc_sent'] = ['yes','no']

In [17]:
def most_similar_answer(a,answer_set):
    a = a.strip().replace(' ', '')
    if(a in answer_set):
        return a
    dis = [Levenshtein.distance(a,x) for x in answer_set]
    idx = np.argmin(dis)
    return answer_set[idx]

In [26]:
def most_similar_answer(a,answer_set):
        a = a.strip().replace(' ', '')
        if(a in answer_set):
            return a
        dis = [Levenshtein.distance(a,x) for x in answer_set]
        idx = np.argmin(dis)
        return answer_set[idx]

out_dict = {i : [] for i in tasks}
out_dict["ner"] = {i : [] for i in ["matscholar", "sofc_token", "sc_comics"]}
out_dict["pc"] = {i : [] for i in ["glass_non_glass"]}
out_dict["sf"] = {i : [] for i in ["sofc_token"]}
out_dict["ee"] = {i : [] for i in ["sc_comics"]}
out_dict["re"] = {i : [] for i in ["structured_re", "sc_comics"]}
out_dict["sar"] = {i : [] for i in ["synthesis_actions"]}
out_dict["sc"] = {i : [] for i in ["sofc_sent"]}
out_dict["qna"] = {i : [] for i in ["squad"]}

for _,sent in enumerate(valfile):
    task = None
    dataset = None
    system = sent['system']
    output = predictions[_]
    answer = sent['answer']
    if 'SOFC' in system:
        task = 'sc'
        dataset = 'sofc_sent'
    elif 'named entity' in system:
        task = 'ner'
        if 'b-dsc' in system:
            dataset = 'matscholar'
        elif 'b-device' in system:
            dataset = 'sofc_token'
        elif 'doping' in system:
            dataset = 'sc_comics'
    elif 'synthesis action' in system:
        task = 'sar'
        dataset = 'synthesis_actions'
    elif 'slots' in system:
        task = 'sf'
        dataset = 'sofc_token'
    elif 'event, identify the roles of the arguments' in system:
        task = 'ee'
        dataset = 'sc_comics'
    elif 'extract relation' in system:
        task = 're'
        for ds in kw[task]:
            if kw[task][ds][0] in system:
                dataset = ds
                break   
    elif 'inorganic glass' in system:
        task = 'pc'
        dataset = 'glass_non_glass'
    else:
        task = 'qna'
        dataset = 'squad'
    
    assert task != None
    assert dataset != None
    
    if task in ["ner", "sf", "ee", "sar"]:
        answer = [i.split(" : ") for i in answer.lower().split('\n')]
        try:
            temp = [[j.strip().lower() for j in i.split(":")] for i in output.split('\n')]
            temp = [i for i in temp if len(i) == 2]
            output = temp
        except:
            print(f"for tc {_}, output pattern mismatched. {output}")
    else:
        answer = answer.strip().lower()
        output = output.strip().lower()

    out_dict[task][dataset].append((answer, output))

scores = {i : [] for i in tasks}
scores["ner"] = {i : (0,0) for i in ["matscholar", "sofc_token", "sc_comics"]}
scores["pc"] = {i : (0,0) for i in ["glass_non_glass"]}
scores["sf"] = {i : (0,0) for i in ["sofc_token"]}
scores["ee"] = {i : (0,0) for i in ["sc_comics"]}
scores["re"] = {i : (0,0) for i in ["structured_re", "sc_comics"]}
scores["sar"] = {i : (0,0) for i in ["synthesis_actions"]}
scores["sc"] = {i : (0,0) for i in ["sofc_sent"]}
scores["qna"] = {i : (0,0) for i in ["squad"]}

def evaluate(task, dataset):
    all_gt = []
    all_pred = []
    for gt, pred in out_dict[task][dataset]:
        dict = {}
        for word, entity in gt:
            dict[word] = [entity, 'O']
                
        for word, entity in pred:
            to_put = most_similar_answer(entity, kw[task][dataset])
            if word in dict:
                dict[word][1] = to_put
            else:
                dict[word] = ['O', to_put]
                
        for i in dict.values():
            all_gt.append(i[0])
            all_pred.append(i[1])

    micro_f1 = f1_score(all_gt, all_pred, average='micro', labels = list(kw[task][dataset]))
    macro_f1 = f1_score(all_gt, all_pred, average='macro', labels = list(kw[task][dataset]))
    scores[task][dataset] = micro_f1, macro_f1
    return micro_f1, macro_f1

def evaluate_pc(dataset):
    out_dict['pc'][dataset] = [(most_similar_answer(i[0], kw['pc'][dataset]), most_similar_answer(i[1], kw['pc'][dataset])) for i in out_dict['pc'][dataset]]

    micro_f1 = f1_score(*list(zip(*out_dict['pc'][dataset])), average='micro', labels = list(kw['pc'][dataset]))
    macro_f1 = f1_score(*list(zip(*out_dict['pc'][dataset])), average='macro', labels = list(kw['pc'][dataset]))
    scores['pc'][dataset] = micro_f1, macro_f1
    return micro_f1, macro_f1

def evaluate_sc(dataset):
    out_dict['sc'][dataset] = [(most_similar_answer(i[0], kw['sc'][dataset]), most_similar_answer(i[1], kw['sc'][dataset])) for i in out_dict['sc'][dataset]]
    
    micro_f1 = f1_score(*list(zip(*out_dict['sc'][dataset])), average='micro', labels = list(kw['sc'][dataset]))
    macro_f1 = f1_score(*list(zip(*out_dict['sc'][dataset])), average='macro', labels = list(kw['sc'][dataset]))
    scores['sc'][dataset] = micro_f1, macro_f1
    return micro_f1, macro_f1

def evaluate_re(dataset):
    out_dict['re'][dataset] = [(most_similar_answer(i[0], kw['re'][dataset]), most_similar_answer(i[1], kw['re'][dataset])) for i in out_dict['re'][dataset]]
    
    micro_f1 = f1_score(*list(zip(*out_dict['re'][dataset])), average='micro', labels = list(kw['re'][dataset]))
    macro_f1 = f1_score(*list(zip(*out_dict['re'][dataset])), average='macro', labels = list(kw['re'][dataset]))
    scores['re'][dataset] = micro_f1, macro_f1
    return micro_f1, macro_f1

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))
    
def get_tokens(s):
    if not s: return []
    return normalize_answer(s).split()


def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def get_raw_scores(dataset):
    exact_scores = []
    f1_scores = []
    for article in dataset:
        gold_answers = [article[0] if normalize_answer(article[0]) else None]
        if not gold_answers[0]:
            print("sad")
            gold_answers = ['']
        a_pred = article[1]
        exact_scores.append(max(compute_exact(a, a_pred) for a in gold_answers))
        f1_scores.append(max(compute_f1(a, a_pred) for a in gold_answers))
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    assert len(exact_scores) == len(f1_scores)
    return collections.OrderedDict([
        ('exact', 1.0 * sum(k for k in exact_scores) / total),
        ('f1', 1.0 * sum(k for k in f1_scores) / total),
        ('total', total),
    ])

def most_similar_answer(a,answer_set):
    a = a.strip().replace(' ', '')
    if(a in answer_set):
        return a
    dis = [Levenshtein.distance(a,x) for x in answer_set]
    idx = np.argmin(dis)
    return answer_set[idx]

def evaluate_qna(dataset):
    exact_raw, f1_raw = get_raw_scores(out_dict['qna'][dataset])
    out_eval = make_eval_dict(exact_raw, f1_raw)
    scores['qna'][dataset] = out_eval['f1'], out_eval['exact']
    
    return out_eval

evaluate_qna('squad')
# evaluate('ner', 'matscholar')
# evaluate('ner', 'sofc_token')
# evaluate('ner', 'sc_comics')
# evaluate('sar', 'synthesis_actions')
# evaluate('ee', 'sc_comics')
# evaluate('sf', 'sofc_token')

# evaluate_sc('sofc_sent')
# evaluate_pc('glass_non_glass')
# evaluate_re('structured_re')
# evaluate_re('sc_comics')


OrderedDict([('exact', 0.7261794634597595),
             ('f1', 0.8554270623306843),
             ('total', 1081)])

In [48]:
for _,sent in enumerate(valfile):
    task = None
    dataset = None
    system = sent['system']
    output = predictions[_]
    answer = sent['answer']
    if 'SOFC' in system:
        task = 'sc'
        dataset = 'sofc_sent'
    elif 'named entity' in system:
        task = 'ner'
        if 'b-dsc' in system:
            dataset = 'matscholar'
        elif 'b-device' in system:
            dataset = 'sofc_token'
        elif 'doping' in system:
            dataset = 'sc_comics'
    elif 'synthesis action' in system:
        task = 'sar'
        dataset = 'synthesis_actions'
    elif 'slots' in system:
        task = 'sf'
        dataset = 'sofc_token'
    elif 'event, identify the roles of the arguments' in system:
        task = 'ee'
        dataset = 'sc_comics'
    elif 'extract relation' in system:
        task = 're'
        for ds in kw[task]:
            if kw[task][ds][0] in system:
                dataset = ds
                break   
    elif 'inorganic glass' in system:
        task = 'pc'
        dataset = 'glass_non_glass'

    assert task != None
    assert dataset != None
    
    if task in ["ner", "sf", "ee", "sar"]:
        answer = [i.split(" : ") for i in answer.lower().split('\n')]
        try:
            temp = [[j.strip().lower() for j in i.split(":")] for i in output.split('\n')]
            temp = [i for i in temp if len(i) == 2]
            output = temp
        except:
            print(f"for tc {_}, output pattern mismatched. {output}")
    else:
        answer = answer.strip().lower()
        output = output.strip().lower()

    out_dict[task][dataset].append((answer, output))

In [50]:
scores = {i : [] for i in tasks}
scores["ner"] = {i : (0,0) for i in ["matscholar", "sofc_token", "sc_comics"]}
scores["pc"] = {i : (0,0) for i in ["glass_non_glass"]}
scores["sf"] = {i : (0,0) for i in ["sofc_token"]}
scores["ee"] = {i : (0,0) for i in ["sc_comics"]}
scores["re"] = {i : (0,0) for i in ["structured_re", "sc_comics"]}
scores["sar"] = {i : (0,0) for i in ["synthesis_actions"]}
scores["sc"] = {i : (0,0) for i in ["sofc_sent"]}

In [61]:
def evaluate(task, dataset):
    all_gt = []
    all_pred = []
    for gt, pred in out_dict[task][dataset]:
        dict = {}
        for word, entity in gt:
            dict[word] = [entity, 'O']
                
        for word, entity in pred:
            to_put = most_similar_answer(entity, kw[task][dataset])
            if word in dict:
                dict[word][1] = to_put
            else:
                dict[word] = ['O', to_put]
                
        for i in dict.values():
            all_gt.append(i[0])
            all_pred.append(i[1])

    micro_f1 = f1_score(all_gt, all_pred, average='micro', labels = list(kw[task][dataset]))
    macro_f1 = f1_score(all_gt, all_pred, average='macro', labels = list(kw[task][dataset]))
    scores[task][dataset] = micro_f1, macro_f1
    return micro_f1, macro_f1

def evaluate_pc(dataset):
    out_dict['pc'][dataset] = [(most_similar_answer(i[0], kw['pc'][dataset]), most_similar_answer(i[1], kw['pc'][dataset])) for i in out_dict['pc'][dataset]]

    micro_f1 = f1_score(*list(zip(*out_dict['pc'][dataset])), average='micro', labels = list(kw['pc'][dataset]))
    macro_f1 = f1_score(*list(zip(*out_dict['pc'][dataset])), average='macro', labels = list(kw['pc'][dataset]))
    scores['pc'][dataset] = micro_f1, macro_f1
    return micro_f1, macro_f1

def evaluate_sc(dataset):
    out_dict['sc'][dataset] = [(most_similar_answer(i[0], kw['sc'][dataset]), most_similar_answer(i[1], kw['sc'][dataset])) for i in out_dict['sc'][dataset]]
    
    micro_f1 = f1_score(*list(zip(*out_dict['sc'][dataset])), average='micro', labels = list(kw['sc'][dataset]))
    macro_f1 = f1_score(*list(zip(*out_dict['sc'][dataset])), average='macro', labels = list(kw['sc'][dataset]))
    scores['sc'][dataset] = micro_f1, macro_f1
    return micro_f1, macro_f1

def evaluate_re(dataset):
    out_dict['re'][dataset] = [(most_similar_answer(i[0], kw['re'][dataset]), most_similar_answer(i[1], kw['re'][dataset])) for i in out_dict['re'][dataset]]
    
    micro_f1 = f1_score(*list(zip(*out_dict['re'][dataset])), average='micro', labels = list(kw['re'][dataset]))
    macro_f1 = f1_score(*list(zip(*out_dict['re'][dataset])), average='macro', labels = list(kw['re'][dataset]))
    scores['re'][dataset] = micro_f1, macro_f1
    return micro_f1, macro_f1

evaluate('ner', 'matscholar')
evaluate('ner', 'sofc_token')
evaluate('ner', 'sc_comics')
evaluate('sar', 'synthesis_actions')
evaluate('ee', 'sc_comics')
evaluate('sf', 'sofc_token')

evaluate_sc('sofc_sent')
evaluate_pc('glass_non_glass')
evaluate_re('structured_re')
evaluate_re('sc_comics')

/home/cse/btech/cs1200389/.conda/envs/matllama-inference/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.9904306220095693, 0.9924803055621867)

In [65]:
df = pd.DataFrame.from_dict({(i,j): scores[i][j] for i in scores.keys() for j in scores[i].keys()},orient='index', columns = ['micro-f1', 'macro-f1'])
df = df.apply(lambda x: 100 * round(x, 5))

# print()
# print(args.pred_file)
# print(df.to_string())
# print()

In [72]:
daddy_df = [df, df, df]

In [74]:
from functools import reduce


In [77]:
df_sum = reduce(lambda x, y: x.add(y, fill_value=0), daddy_df)
df_mean = df_sum / len(daddy_df)
df_concat = pd.concat(daddy_df)
df_std = df_concat.groupby(df_concat.index).std()

df_mean.columns = pd.MultiIndex.from_tuples([(i, f"mean") for i in df_mean.columns])
df_std.columns = pd.MultiIndex.from_tuples([(i, f"std") for i in df_std.columns])

combined_df = pd.concat([df_mean, df_std], axis=1)
combined_df = combined_df.sort_index(axis=1).apply(lambda x : round(x,5))


print(f"===eval over average of {args.num_seeds} runs===")
print(combined_df.to_string())

prefix = args.checkpoint.split('/')[-1]

now = datetime.datetime.now()
datetimestring = now.strftime("%y-%m-%d-%H-%M")

combined_df.to_csv(f'csvs/{prefix}-{datetimestring}.csv')

AttributeError: 'Namespace' object has no attribute 'num_seeds'

In [78]:
combined_df

macro-f1      micro-f1     
                             mean  std     mean  std
(ner, matscholar)          80.555  0.0   85.442  0.0
(ner, sofc_token)          76.826  0.0   88.260  0.0
(ner, sc_comics)           88.974  0.0   91.752  0.0
(pc, glass_non_glass)      92.945  0.0   94.000  0.0
(sf, sofc_token)           68.625  0.0   78.169  0.0
(ee, sc_comics)            87.843  0.0   91.000  0.0
(re, structured_re)       100.000  0.0  100.000  0.0
(re, sc_comics)            99.248  0.0   99.043  0.0
(sar, synthesis_actions)   94.737  0.0   96.682  0.0
(sc, sofc_sent)            78.980  0.0   93.978  0.0